#### Implement masked attention where only tokens before the current input word can be accessed 

In [1]:
import torch 

In [ ]:

attn_weights= torch.tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]]) 
# dim: 6x6 

# attention weights for each of the 6 words with respect to other words (Think of a comparison matrix)

To create a masking effect, we get the regular attention but multiply with a tril matrix so that only a lower triangular matrix remains

In [4]:
context_length= attn_weights.shape[0]   # get number of words we have in input 
mask_simple= torch.tril(torch.ones(context_length, context_length)) # create a lower triangular matrix of a [context_length] x [context_lenth] matrix
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [6]:
masked_simple=attn_weights * mask_simple    # point wise multiplication 
masked_simple

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]])

Now for each word, we only have access to the attention weight of the previous words 

We we need to normalize the attention scores again --> each row adds up to 1 

In [7]:
row_sums= masked_simple.sum(dim=-1, keepdim=True) 
masked_simple_norm= masked_simple/row_sums  # element wise row 
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5516, 0.4484, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3801, 0.3097, 0.3102, 0.0000, 0.0000, 0.0000],
        [0.2759, 0.2461, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1985, 0.1887, 0.1970, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]])


**Note that** by computing the softmax on attention weights that are calculated and normalized with all tokens again, the effect of future words are nullified. So our attention weights after normalization is like we never got information from future tokens in the first place.   
- Softmax does probability distribution again on a smaller set of values 

We can also apply **drop out** to drop out random token's attention weights to prevent overfitting. Like how Neural nets overly rely on a certain hidden layer node. Drop outs also scale up attention weights to ensure the balance of attention 